In [1]:
import numpy as np

def round_to_m_digits(number, m):
    """
    Округляет число до m знаков после запятой.
    """
    return round(number, m)

def gaussian_elimination_with_partial_pivoting(A, b):
    n = len(b)

    # Прямой ход
    for k in range(n-1):
        # Поиск опорного элемента в k-м столбце
        maxindex = abs(A[k:,k]).argmax() + k
        if A[maxindex, k] == 0:
            raise ValueError("Матрица вырождена.")
        # Перестановка строк
        if maxindex != k:
            A[[k,maxindex]] = A[[maxindex, k]]
            b[[k,maxindex]] = b[[maxindex, k]]
        # Обнуление элементов в k-м столбце ниже опорного
        for row in range(k+1, n):
            multiplier = A[row][k]/A[k][k]
            A[row][k] = 0
            A[row][k+1:] = A[row][k+1:] - multiplier*A[k][k+1:]
            b[row] = b[row] - multiplier*b[k]

    # Обратный ход
    x = np.zeros(n)
    for k in range(n-1, -1, -1):
        x[k] = (b[k] - np.dot(A[k][k+1:], x[k+1:])) / A[k][k]
    return x

def calculate_elements(n, M, m, t):
    """
    Вычисляет элементы матрицы A и вектора b для заданных параметров,
    с округлением до m знаков после запятой.
    """
    # Вычисление q на основе M
    q = 0.993 + ((-1)**M) * M * 10**(-4)
    q = round_to_m_digits(q, m)  # Округление q до m знаков после запятой

    # Инициализация матрицы A и вектора b
    A = np.zeros((n, n))
    b = np.zeros(n)

    # Заполнение A и b соответствующими значениями, округленными до m знаков после запятой
    for i in range(n):
        for j in range(n):
            A[i, j] = round_to_m_digits(q**(i+1), m) if i != j else round_to_m_digits(q**(i+1) + t, m)
        b[i] = round_to_m_digits(q**M * (n - i), m)

    return A, b

# Параметры из задания
n = 50  # Размер матрицы и вектора
M = 1   # Задано в задаче
m = 6   # Количество знаков после запятой для округления
t_values = [0.0001, 1, 10000]  # Значения параметра t

# Решение системы для каждого значения t и сохранение решений
solutions = {}
for t in t_values:
    A, b = calculate_elements(n, M, m, t)
    try:
        solution = gaussian_elimination_with_partial_pivoting(A, b)
        solution = np.around(solution, m)  # Округление вектора решений
        solutions[t] = solution
    except ValueError as e:
        solutions[t] = str(e)

# Вывод решений (показаны только первые и последние 5 элементов для краткости)
for t in solutions:
    print(f"Решения для t = {t}:")
    print("Первые 5 элементов:", solutions[t][:5])
    print("Последние 5 элементов:", solutions[t][-5:])


Решения для t = 0.0001:
Первые 5 элементов: [196557.102989 188757.466393 180942.425892 173112.58556  165267.64336 ]
Последние 5 элементов: [-167982.58472  -176366.663612 -184761.313812 -193167.139394
 -201583.536283]
Решения для t = 1:
Первые 5 элементов: [20.354471 19.569546 18.783117 17.995241 17.205891]
Последние 5 элементов: [-16.291178 -17.133186 -17.976226 -18.820358 -19.665522]
Решения для t = 10000:
Первые 5 элементов: [0.004952 0.004853 0.004754 0.004654 0.004555]
Последние 5 элементов: [4.87e-04 3.88e-04 2.89e-04 1.90e-04 9.00e-05]
